# Street End Parameter Tuning

This notebook helps tune parameters for the street end finder algorithm.

In [2]:
import sys
from main import StreetEndFinder
from render_map import StreetEndRenderer
import folium
from IPython.display import display, HTML
import pandas as pd
import geopandas as gpd
from datetime import datetime
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Test parameters
params = {
    'city': 'Skokie, USA',
    'threshold_distance': 10,     # meters from water
    'buffer_distance': 100,       # meters around water to search for streets
    'min_point_spacing': 25,      # minimum meters between points
    'find_street_ends': True,     # whether to find street ends
    'enrich_data': True,         # whether to enrich with additional data
    'update_existing': True      # whether to update existing analysis
}

# Display current parameters
pd.DataFrame([params]).T

,0
city,"Skokie, USA"
threshold_distance,10
buffer_distance,100
min_point_spacing,25
find_street_ends,True
enrich_data,True
update_existing,True


# Third cell - analysis function and execution


In [5]:
def analyze_single_city(params):
    """Run analysis for a single city with given parameters"""
    global water_features, near_water  # Make these global so other cells can access them
    
    # Stage 1: Find street ends
    finder = StreetEndFinder(
        params['city'],
        threshold_distance=params['threshold_distance'],
        find_street_ends=params['find_street_ends'],
        enrich_data=params['enrich_data']
    )
    water_features, near_water = finder.process()
    
    # Create summary file structure
    summary = {
        'type': 'FeatureCollection',
        'name': 'street_ends',
        'properties': {
            'city_name': params['city'],
            'total_street_ends': len(near_water),
            'analysis_date': datetime.now().isoformat()
        },
        'features': near_water
    }
    
    # Save summary file
    with open('temp_summary.json', 'w') as f:
        json.dump(summary, f)
    
    # Create visualization
    renderer = StreetEndRenderer(
        'street_ends_near_river.geojson',
        params['city'],
        summary_file='temp_summary.json',
        threshold_distance=params['threshold_distance']
    )
    renderer.render('temp_map.html')
    
    return water_features, near_water, renderer

# Run analysis and store results
water_features, near_water, renderer = analyze_single_city(params)

# Display the map in the notebook
display(IFrame(src='temp_map.html', width='100%', height=600))


TypeError: Object of type Point is not JSON serializable

In [13]:
def get_statistics(near_water, water_features):
    """Calculate and display statistics about the results"""
    import numpy as np
    
    if near_water is None or water_features is None:
        return pd.DataFrame({
            'Value': ['No data available']
        }, index=['Status'])
    
    try:
        stats = {
            'Total Water Features': len(water_features),
            'Total Street Ends Near Water': len(near_water),
            'Average Distance to Water': np.mean([p['properties']['distance_to_water'] for p in near_water]) if near_water else 0,
            'Min Distance to Water': min([p['properties']['distance_to_water'] for p in near_water]) if near_water else 0,
            'Max Distance to Water': max([p['properties']['distance_to_water'] for p in near_water]) if near_water else 0
        }
        return pd.DataFrame([stats]).T.rename(columns={0: 'Value'})
    except Exception as e:
        print(f"Error calculating statistics: {str(e)}")
        return pd.DataFrame({
            'Value': ['Error calculating statistics']
        }, index=['Status'])

# Display statistics
if 'near_water' in locals() and 'water_features' in locals():
    display(get_statistics(near_water, water_features))
else:
    display(pd.DataFrame({
        'Value': ['Analysis not yet run']
    }, index=['Status']))

,Value
Status,Analysis not yet run


In [ ]:
def plot_distance_distribution(near_water):
    """Plot distribution of distances to water"""
    distances = [point['properties']['distance_to_water'] for point in near_water]
    
    plt.figure(figsize=(10, 6))
    sns.histplot(distances, bins=20)
    plt.title('Distribution of Distances to Water')
    plt.xlabel('Distance (meters)')
    plt.ylabel('Count')
    plt.show()

plot_distance_distribution(near_water)

In [ ]:
def plot_distance_distribution(near_water):
    """Plot distribution of distances to water"""
    distances = [point['properties']['distance_to_water'] for point in near_water]
    
    plt.figure(figsize=(10, 6))
    sns.histplot(distances, bins=20)
    plt.title('Distribution of Distances to Water')
    plt.xlabel('Distance (meters)')
    plt.ylabel('Count')
    plt.show()

plot_distance_distribution(near_water)